In [19]:
# dataset
# https://www.kaggle.com/datasets/mathurinache/math-dataset
# !kaggle datasets download -d mathurinache/math-dataset

In [2]:
from IPython.display import Latex, display
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(DEVICE)

cpu


In [4]:
problem_df = pd.read_json('./math_dataset_v1_train.json')
problem_df.head()

,question,answer
0,How many vertical asymptotes does the graph of...,The denominator of the rational function facto...
1,What is the positive difference between $120\%...,One hundred twenty percent of 30 is $120\cdot3...
2,Find $x$ such that $\lceil x \rceil + x = \dfr...,"First, we note that $x$ must be positive, sinc..."
3,Evaluate $i^5+i^{-25}+i^{45}$.,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. ...
4,"If $2^8=4^x$, what is the value of $x$?",Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Si...


In [14]:
sample = problem_df.loc[1]


print("Question:\n")
display(Latex(sample['questions']))
print('Solution:\n')
display(Latex(sample['answers']))

Question:



<IPython.core.display.Latex object>

Solution:



<IPython.core.display.Latex object>

In [3]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [4]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [5]:
class MathXDataset(Dataset):
    def __init__(self, problems: pd.DataFrame):
        self.Q_A = tokenizer(text=problems['questions'].tolist(), 
                             text_target=problems['answers'].tolist(), 
                             padding=True, 
                             truncation=True, 
                             return_tensors='pt')

    def __len__(self):
        return len(self.Q_A)

    def __getitem__(self, idx):
        return self.Q_A[idx]

In [64]:
dataset = MathXDataset(problem_df)

In [75]:
dataset[0][0]

TypeError: 'tokenizers.Encoding' object is not subscriptable